In [17]:
%load_ext autoreload
%autoreload 2

import config      #python file with API credentials for authentication
import matplotlib
import tweepy
import twitter
import requests
import json
import time 
import pandas as pd
import os
import sys
import gzip
import numpy as np

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
users_data = pd.read_csv('data_2023_4_6.csv')
users_data

,Unnamed: 0,id,created_at,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,lang,...,possibly_sensitive,hashtags,user_mentions,urls,user_id,screen_name,description,user_location,tweet_id,author_id
0,0,1623189880787591168,Wed Feb 08 05:19:59 +0000 2023,「情報セキュリティ10大脅威 2023」が公開されました！\n昨年は圏外だった脅威がランクイ...,False,False,False,2,0,ja,...,False,[],[],['https://www.ipa.go.jp/security/vuln/10threat...,1238329176853114880,NWU_CSIRT,奈良女子大学CSIRTです。\n情報セキュリティに関する注意喚起や当CSIRTの活動について...,奈良女子大学,1623189880787591168,1238329176853114880
1,1,1613699397384294400,Fri Jan 13 00:48:12 +0000 2023,明日から大学入学共通テストですね。朝は寒い日が続いています。受験生のみなさん、暖かくして臨ん...,False,False,False,0,0,ja,...,NaN,[],[],[],1238329176853114880,NWU_CSIRT,奈良女子大学CSIRTです。\n情報セキュリティに関する注意喚起や当CSIRTの活動について...,奈良女子大学,1613699397384294400,1238329176853114880
2,2,1602185448159207424,Mon Dec 12 06:15:52 +0000 2022,今年度も標的型攻撃メール訓練を実施しますので、ご協力をお願いします。\nまた、不審なメール等...,False,False,False,1,1,ja,...,False,[],[],['https://www.nara-wu.ac.jp/csirt/event/202212...,1238329176853114880,NWU_CSIRT,奈良女子大学CSIRTです。\n情報セキュリティに関する注意喚起や当CSIRTの活動について...,奈良女子大学,1602185448159207424,1238329176853114880
3,3,1600347419677687809,Wed Dec 07 04:32:12 +0000 2022,【注意喚起】WordPressに複数の脆弱性が確認されています。\n詳細はこちら→https...,False,False,False,0,0,ja,...,False,[],[],['https://jvn.jp/jp/JVN09409909/'],1238329176853114880,NWU_CSIRT,奈良女子大学CSIRTです。\n情報セキュリティに関する注意喚起や当CSIRTの活動について...,奈良女子大学,1600347419677687809,1238329176853114880
4,4,1598146454794956800,Thu Dec 01 02:46:21 +0000 2022,大学生協アプリに限らず、アプリをインストールする際は危険なアプリでないか注意しましょう。\n...,False,False,False,0,0,ja,...,False,[],[],['https://10mtv.jp/pc/column/article.php?colum...,1238329176853114880,NWU_CSIRT,奈良女子大学CSIRTです。\n情報セキュリティに関する注意喚起や当CSIRTの活動について...,奈良女子大学,1598146454794956800,1238329176853114880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,855,1286259036493668360,Thu Jul 23 11:17:06 +0000 2020,Today we introduce Juraj Smeriga's thesis on t...,False,False,False,2,7,en,...,False,"['MachineLearning', 'phishing']",[],['https://muni.cz/go/5eb013'],2787968336,csirtmu,We are cybersecurity team of Masaryk Universit...,"Brno, Czech Republic",1286259036493668360,2787968336
856,856,1285911310148960259,Wed Jul 22 12:15:22 +0000 2020,"In July and August, we will present a series o...",False,False,False,1,6,en,...,False,['theses'],['csirtmu'],[],2787968336,csirtmu,We are cybersecurity team of Masaryk Universit...,"Brno, Czech Republic",1285911310148960259,2787968336
857,857,1283726233910546433,Thu Jul 16 11:32:39 +0000 2020,#KYPO cyber range hosted an #online training f...,False,False,False,6,15,en,...,False,"['KYPO', 'online', 'cybersecurity', 'students'...","['KyberSoutez', 'FI_MUNI']",[],2787968336,csirtmu,We are cybersecurity team of Masaryk Universit...,"Brno, Czech Republic",1283726233910546433,2787968336
858,858,1281574719808995329,Fri Jul 10 13:03:18 +0000 2020,Martin Husák's and Tomas Jirsik's SoK paper on...,False,False,False,1,3,en,...,False,"['network', 'security', 'ares2020']",['ARES_Conference'],['https://ares-conference.eu/registration-all-...,2787968336,csirtmu,We are cybersecurity team of Masaryk Universit...,"Brno, Czech Republic",1281574719808995329,2787968336


In [20]:
users_data['tweet_id']

0      1623189880787591168
1      1613699397384294400
2      1602185448159207424
3      1600347419677687809
4      1598146454794956800
              ...         
855    1286259036493668360
856    1285911310148960259
857    1283726233910546433
858    1281574719808995329
859    1619227881024131072
Name: tweet_id, Length: 860, dtype: int64

In [21]:
tweet_fields = 'author_id,conversation_id,created_at,entities,geo,id,lang,public_metrics,referenced_tweets,source,text,withheld' # specify tweet fields to extract
user_fields = 'created_at,description,entities,id,location,name,public_metrics,url,username,verified,withheld' # specify user fields to extract
place_fields = 'contained_within,country,country_code,full_name,geo,id,name,place_type' # specify place fields to extract
media_fields = 'duration_ms,height,media_key,preview_image_url,type,url,width' # specify media fields to extract
poll_fields = 'duration_minutes,end_datetime,id,options,voting_status' # specify poll fields to extract
max_tweet_ids = 100 # maximum number of tweet ids to include in a single API call


In [22]:
bearer_token = config.bearer_token
headers = {"Authorization": "Bearer {}".format(bearer_token)}
# create an empty DataFrame to store the tweet data
tweet_data_df = pd.DataFrame()
# Define a function to create a Pandas DataFrame from the tweet data
def create_dataframe(tweet_data):
    # Create an empty list to store the tweet data
    tweets = []
    # Iterate through each tweet in the tweet data
    for tweet in tweet_data:
        # Append the tweet data to the list
        tweets.append(tweet)
    # Convert the list of tweets to a Pandas DataFrame
    df = pd.DataFrame(tweets)
    return df

In [23]:
# reset the index of the DataFrame
tweet_data_df = tweet_data_df.reset_index(drop=True)

In [2]:
#add new dataset made on 22nd march
tweets_150_per_author_no_conv_id = pd.read_excel('tweets_150_per_author_no_conv.xlsx')

In [3]:
tweets_150_per_author_no_conv_id

,Unnamed: 0.1,Unnamed: 0,id,created_at,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,...,coordinates,possibly_sensitive,hashtags,user_mentions,urls,user_id,screen_name,description,user_location,tweet_id
0,0,3545,1619227881024131072,Sat Jan 28 06:56:25 +0000 2023,Verizon admitted to me one of their marketing ...,False,False,False,12,19,...,NaN,0.0,[],[],[],37888607,billtoulas,"Tech writer, infosec news reporter @BleepinCom...","Nicosia, Cyprus",1619227881024131072
1,1,10571,1176602940615909120,Tue Sep 24 21:02:55 +0000 2019,Опубликовали первое в России масштабное сравне...,False,False,False,7,19,...,NaN,0.0,[],"['CiscoSecurity', 'Fortinet', 'PaloAltoNtwks',...",['https://go.shr.lc/2mMwdQx'],137795216,IlyaShabanov,The founder and CEO of AM Media LLC,Moscow,1176602940615909376
2,2,10591,1113834605289853056,Thu Apr 04 16:03:58 +0000 2019,"ГосСОПКА: что такое, зачем нужна и как устроен...",False,False,False,4,9,...,NaN,0.0,[],['Anti_Malware'],['https://go.shr.lc/2Uo0jsJ'],137795216,IlyaShabanov,The founder and CEO of AM Media LLC,Moscow,1113834605289852929
3,3,10673,989593024446509056,Thu Apr 26 19:52:17 +0000 2018,Круто! Теперь в Gmail на письма можно установи...,False,False,False,6,9,...,NaN,0.0,[],[],['https://go.shr.lc/2r1qSTT'],137795216,IlyaShabanov,The founder and CEO of AM Media LLC,Moscow,989593024446509056
4,4,10607,1074592431637709056,Mon Dec 17 09:09:35 +0000 2018,Обзор АПКШ «Континент» 3.9 – многофункциональн...,False,False,False,2,8,...,NaN,0.0,[],['Anti_Malware'],['https://go.shr.lc/2Qy7M7m'],137795216,IlyaShabanov,The founder and CEO of AM Media LLC,Moscow,1074592431637708800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,1565,6456,1456639341795610880,Fri Nov 05 15:07:36 +0000 2021,Serwery #GitLab są atakowane #DDoS. Atakujący ...,False,False,False,0,3,...,NaN,0.0,"['GitLab', 'DDoS']",[],['https://www.rapid7.com/blog/post/2021/11/01/...,1290574179729891336,CSIRT_MON,CSIRT MON - Zespół Reagowania na Incydenty Bez...,Polska,1456639341795610624
1566,1566,6457,1455842264941568000,Wed Nov 03 10:20:18 +0000 2021,Firma Google wydała listopadowy biuletyn bezpi...,False,False,False,0,3,...,NaN,0.0,['Android'],[],['https://source.android.com/security/bulletin...,1290574179729891336,CSIRT_MON,CSIRT MON - Zespół Reagowania na Incydenty Bez...,Polska,1455842264941568002
1567,1567,6463,1394182085896777984,Mon May 17 06:44:46 +0000 2021,"Płk Łukasz Jędrzejczak, zastępca dyrektora NCB...",False,False,False,0,2,...,NaN,0.0,[],[],['https://cyberdefence24.pl/walka-o-ekspertow-...,1290574179729891336,CSIRT_MON,CSIRT MON - Zespół Reagowania na Incydenty Bez...,Polska,1394182085896777729
1568,1568,6449,1470695622881811968,Tue Dec 14 10:02:15 +0000 2021,Najnowsza aktualizacja dla produktów Apple - i...,False,False,False,0,1,...,NaN,0.0,"['apple', 'ios', 'vulnerability']",[],"['http://m.in', 'https://support.apple.com/pl-...",1290574179729891336,CSIRT_MON,CSIRT MON - Zespół Reagowania na Incydenty Bez...,Polska,1470695622881812480


continue with this data - you need to add conv id

In [24]:
# set up the API endpoint
url = 'https://api.twitter.com/2/tweets'

In [25]:
tweet_ids = users_data['tweet_id']
tweet_ids

0      1623189880787591168
1      1613699397384294400
2      1602185448159207424
3      1600347419677687809
4      1598146454794956800
              ...         
855    1286259036493668360
856    1285911310148960259
857    1283726233910546433
858    1281574719808995329
859    1619227881024131072
Name: tweet_id, Length: 860, dtype: int64

In [26]:
# create an empty dataframe to store the results
tweets_df = pd.DataFrame()
max_tweet_ids = 100
# loop through the tweet ids in batches and extract the required data
for i in range(0, len(tweet_ids), max_tweet_ids):
    # create a comma-separated list of tweet ids to include in the API call
    tweet_ids_str = ','.join(tweet_ids[i:i+max_tweet_ids].astype(str))
    # set up the API endpoint
    url = 'https://api.twitter.com/2/tweets'
    # set up the API parameters
    params = {
        
        'ids': tweet_ids_str,
        'tweet.fields': tweet_fields,
        'user.fields': user_fields,
        'place.fields': place_fields,
        'media.fields': media_fields,
        'poll.fields': poll_fields,
    }
    
    # set up the API headers with the authentication credentials
    headers = {
        'Authorization': f'Bearer {bearer_token}'
    }
    
    # make the API call
    response = requests.get(url, headers=headers, params=params)
    
    for line in response.iter_lines():
        if line:
            # Decode the line and convert it to a dictionary
            tweet_dict = json.loads(line.decode("utf-8"))
            # Print the tweet data to the console
            # print(json.dumps(tweet_dict, indent=4))
            # Append the tweet data to the JSON file
            # save_to_json([tweet_dict], output_file)
            df = create_dataframe([tweet_dict])
            
            # Print the DataFrame to the console

    # convert the response to a JSON object
    response_json = response.json()
    tweet_data = response_json['data']
    tweet_data_df = tweet_data_df.append(pd.json_normalize(tweet_data))

    # # extract the tweet and user data from the JSON object
    # tweet_data = response_json['data']
    # user_data = response_json['includes']['users']
    # place_data = response_json['includes'].get('places', [])
    # media_data = response_json['includes'].get('media', [])
    # poll_data = response_json['includes'].get('polls', [])
    
    # # create a dictionary to map user ids to user data
    # user_dict = {}
    # for user in user_data:
    #     user_dict[user['id']] = user
    
    # # create a dictionary to map place ids to place data
    # place_dict = {}
    # for place in place_data:
    #     place_dict[place['id']] = place
    
    # # create a dictionary to map media keys to media data
    # media_dict = {}
    # for media in media_data:
    #     media_dict[media['media_key']] = media
    
    # # create a dictionary to map poll ids to poll data
    # poll_dict = {}
    # for poll in poll_data:
    #     poll_dict[poll['id']] = poll
    
   

C:\Users\Nastea\AppData\Local\Temp\ipykernel_15944\2452791426.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweet_data_df = tweet_data_df.append(pd.json_normalize(tweet_data))
C:\Users\Nastea\AppData\Local\Temp\ipykernel_15944\2452791426.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweet_data_df = tweet_data_df.append(pd.json_normalize(tweet_data))
C:\Users\Nastea\AppData\Local\Temp\ipykernel_15944\2452791426.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweet_data_df = tweet_data_df.append(pd.json_normalize(tweet_data))
C:\Users\Nastea\AppData\Local\Temp\ipykernel_15944\2452791426.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

In [27]:
tweet_data_df

,edit_history_tweet_ids,created_at,id,conversation_id,text,author_id,lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.urls,referenced_tweets,entities.annotations,entities.hashtags,entities.mentions,geo.coordinates.type,geo.coordinates.coordinates,geo.place_id
0,[1623189880787591168],2023-02-08T05:19:59.000Z,1623189880787591168,1623189880787591168,「情報セキュリティ10大脅威 2023」が公開されました！\n昨年は圏外だった脅威がランクイ...,1238329176853114880,ja,2,0,0,0,3031,"[{'start': 99, 'end': 122, 'url': 'https://t.c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[1613699397384294400],2023-01-13T00:48:12.000Z,1613699397384294400,1613699397384294400,明日から大学入学共通テストですね。朝は寒い日が続いています。受験生のみなさん、暖かくして臨ん...,1238329176853114880,ja,0,0,0,0,281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[1602185448159207424],2022-12-12T06:15:52.000Z,1602185448159207424,1602185448159207424,今年度も標的型攻撃メール訓練を実施しますので、ご協力をお願いします。\nまた、不審なメール等...,1238329176853114880,ja,1,0,1,0,0,"[{'start': 75, 'end': 98, 'url': 'https://t.co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[1600347419677687809],2022-12-07T04:32:12.000Z,1600347419677687809,1600347419677687809,【注意喚起】WordPressに複数の脆弱性が確認されています。\n詳細はこちら→https...,1238329176853114880,ja,0,0,0,0,0,"[{'start': 40, 'end': 63, 'url': 'https://t.co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[1598146454794956800],2022-12-01T02:46:21.000Z,1598146454794956800,1598146454794956800,大学生協アプリに限らず、アプリをインストールする際は危険なアプリでないか注意しましょう。\n...,1238329176853114880,ja,0,0,0,0,0,"[{'start': 45, 'end': 68, 'url': 'https://t.co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,[1286259036493668360],2020-07-23T11:17:06.000Z,1286259036493668360,1286259036493668360,Today we introduce Juraj Smeriga's thesis on t...,2787968336,en,2,0,7,0,0,"[{'start': 210, 'end': 233, 'url': 'https://t....",NaN,"[{'start': 25, 'end': 31, 'probability': 0.509...","[{'start': 64, 'end': 80, 'tag': 'MachineLearn...",NaN,NaN,NaN,NaN
56,[1285911310148960259],2020-07-22T12:15:22.000Z,1285911310148960259,1285911310148960259,"In July and August, we will present a series o...",2787968336,en,1,0,6,0,0,"[{'start': 207, 'end': 230, 'url': 'https://t....",NaN,NaN,"[{'start': 58, 'end': 65, 'tag': 'theses'}]","[{'start': 158, 'end': 166, 'username': 'csirt...",NaN,NaN,NaN
57,[1283726233910546433],2020-07-16T11:32:39.000Z,1283726233910546433,1283726233910546433,#KYPO cyber range hosted an #online training f...,2787968336,en,6,0,15,0,0,"[{'start': 259, 'end': 282, 'url': 'https://t....",NaN,"[{'start': 1, 'end': 4, 'probability': 0.6343,...","[{'start': 0, 'end': 5, 'tag': 'KYPO'}, {'star...","[{'start': 98, 'end': 110, 'username': 'KyberS...",NaN,NaN,NaN
58,[1281574719808995329],2020-07-10T13:03:18.000Z,1281574719808995329,1281574719808995329,Martin Husák's and Tomas Jirsik's SoK paper on...,2787968336,en,1,0,3,1,0,"[{'start': 228, 'end': 251, 'url': 'https://t....",NaN,"[{'start': 0, 'end': 11, 'probability': 0.8439...","[{'start': 124, 'end': 132, 'tag': 'network'},...","[{'start': 204, 'end': 220, 'username': 'ARES_...",NaN,NaN,NaN


In [28]:
tweet_data_df.rename(columns={'text': 'full_text'}, inplace=True)


In [29]:
tweet_data_df.reset_index(inplace=True)
tweet_data_df

,index,edit_history_tweet_ids,created_at,id,conversation_id,full_text,author_id,lang,public_metrics.retweet_count,public_metrics.reply_count,...,public_metrics.quote_count,public_metrics.impression_count,entities.urls,referenced_tweets,entities.annotations,entities.hashtags,entities.mentions,geo.coordinates.type,geo.coordinates.coordinates,geo.place_id
0,0,[1623189880787591168],2023-02-08T05:19:59.000Z,1623189880787591168,1623189880787591168,「情報セキュリティ10大脅威 2023」が公開されました！\n昨年は圏外だった脅威がランクイ...,1238329176853114880,ja,2,0,...,0,3031,"[{'start': 99, 'end': 122, 'url': 'https://t.c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,[1613699397384294400],2023-01-13T00:48:12.000Z,1613699397384294400,1613699397384294400,明日から大学入学共通テストですね。朝は寒い日が続いています。受験生のみなさん、暖かくして臨ん...,1238329176853114880,ja,0,0,...,0,281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,[1602185448159207424],2022-12-12T06:15:52.000Z,1602185448159207424,1602185448159207424,今年度も標的型攻撃メール訓練を実施しますので、ご協力をお願いします。\nまた、不審なメール等...,1238329176853114880,ja,1,0,...,0,0,"[{'start': 75, 'end': 98, 'url': 'https://t.co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,[1600347419677687809],2022-12-07T04:32:12.000Z,1600347419677687809,1600347419677687809,【注意喚起】WordPressに複数の脆弱性が確認されています。\n詳細はこちら→https...,1238329176853114880,ja,0,0,...,0,0,"[{'start': 40, 'end': 63, 'url': 'https://t.co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,[1598146454794956800],2022-12-01T02:46:21.000Z,1598146454794956800,1598146454794956800,大学生協アプリに限らず、アプリをインストールする際は危険なアプリでないか注意しましょう。\n...,1238329176853114880,ja,0,0,...,0,0,"[{'start': 45, 'end': 68, 'url': 'https://t.co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,55,[1286259036493668360],2020-07-23T11:17:06.000Z,1286259036493668360,1286259036493668360,Today we introduce Juraj Smeriga's thesis on t...,2787968336,en,2,0,...,0,0,"[{'start': 210, 'end': 233, 'url': 'https://t....",NaN,"[{'start': 25, 'end': 31, 'probability': 0.509...","[{'start': 64, 'end': 80, 'tag': 'MachineLearn...",NaN,NaN,NaN,NaN
856,56,[1285911310148960259],2020-07-22T12:15:22.000Z,1285911310148960259,1285911310148960259,"In July and August, we will present a series o...",2787968336,en,1,0,...,0,0,"[{'start': 207, 'end': 230, 'url': 'https://t....",NaN,NaN,"[{'start': 58, 'end': 65, 'tag': 'theses'}]","[{'start': 158, 'end': 166, 'username': 'csirt...",NaN,NaN,NaN
857,57,[1283726233910546433],2020-07-16T11:32:39.000Z,1283726233910546433,1283726233910546433,#KYPO cyber range hosted an #online training f...,2787968336,en,6,0,...,0,0,"[{'start': 259, 'end': 282, 'url': 'https://t....",NaN,"[{'start': 1, 'end': 4, 'probability': 0.6343,...","[{'start': 0, 'end': 5, 'tag': 'KYPO'}, {'star...","[{'start': 98, 'end': 110, 'username': 'KyberS...",NaN,NaN,NaN
858,58,[1281574719808995329],2020-07-10T13:03:18.000Z,1281574719808995329,1281574719808995329,Martin Husák's and Tomas Jirsik's SoK paper on...,2787968336,en,1,0,...,1,0,"[{'start': 228, 'end': 251, 'url': 'https://t....",NaN,"[{'start': 0, 'end': 11, 'probability': 0.8439...","[{'start': 124, 'end': 132, 'tag': 'network'},...","[{'start': 204, 'end': 220, 'username': 'ARES_...",NaN,NaN,NaN


In [30]:
tweet_data_df.columns

Index(['index', 'edit_history_tweet_ids', 'created_at', 'id',
       'conversation_id', 'full_text', 'author_id', 'lang',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.impression_count', 'entities.urls', 'referenced_tweets',
       'entities.annotations', 'entities.hashtags', 'entities.mentions',
       'geo.coordinates.type', 'geo.coordinates.coordinates', 'geo.place_id'],
      dtype='object')

In [31]:
tweet_data_df.columns

tweet_data_df['tweet_id'] = tweet_data_df['id'].astype(str)

tweet_data_df['conversation_id'] = tweet_data_df['conversation_id'].astype(str)

tweet_data_df['author_id'] = tweet_data_df['author_id'].astype(str)



In [46]:
# tweet_data_df = pd.read_excel('tweet_data_df_from_27k.xlsx')


In [32]:
#save the 150 tw per user
tweet_data_df.to_excel('data_with_convid_2023_4_6.xlsx', encoding='utf-f8')


C:\Users\Nastea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


reproduce back # tweet_data_df = pd.read_excel('tweet_data_df_from_27k.xlsx')


In [19]:
tweet_data_df = pd.read_excel('tweet_data_df_from_27k.xlsx')


In [20]:
tweet_data_df

,Unnamed: 0,edit_history_tweet_ids,conversation_id,author_id,created_at,full_text,lang,id,entities.urls,public_metrics.retweet_count,...,public_metrics.quote_count,public_metrics.impression_count,entities.mentions,entities.annotations,entities.hashtags,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0,['1005188544216944643'],1005188544216944643,925834897838804992,2018-06-08T20:43:19.000Z,Boletín CSIRT 2018-06-08 https://t.co/30QRd415sD,und,1005188544216944643,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",3,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,['1004835128428908546'],1004835128428908546,925834897838804992,2018-06-07T21:18:58.000Z,Boletín CSIRT 2018-06-07 https://t.co/lBBw4Mtt2d,und,1004835128428908546,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",3,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,['1004470327420825601'],1004470327420825601,925834897838804992,2018-06-06T21:09:22.000Z,Boletín CSIRT 2018-06-06 https://t.co/jYz8n1Gciz,und,1004470327420825601,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",3,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,['1004468583794184194'],1004468583794184194,925834897838804992,2018-06-06T21:02:27.000Z,Update Google Chrome Immediately to Patch a Hi...,en,1004468583794184194,"[{'start': 74, 'end': 97, 'url': 'https://t.co...",1,...,0,0,"[{'start': 102, 'end': 117, 'username': 'TheHa...","[{'start': 7, 'end': 19, 'probability': 0.9137...",NaN,NaN,NaN,NaN,NaN,NaN
4,4,['1003742842756444160'],1003742842756444160,925834897838804992,2018-06-04T20:58:37.000Z,Boletín CSIRT 2018-06-04 https://t.co/kU7OU2LRMA,und,1003742842756444160,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",2,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21705,6,['2775715724'],2775715724,56187579,2009-07-22T10:20:47.000Z,Irish websites compromised with malware code. ...,en,2775715724,NaN,0,...,0,0,NaN,"[{'start': 82, 'end': 84, 'probability': 0.704...",NaN,NaN,NaN,NaN,NaN,NaN
21706,7,['2775652740'],2775652740,56187579,2009-07-22T10:13:14.000Z,RT @markofu: RT @securityninja: OWASP Ireland ...,en,2775652740,NaN,0,...,0,0,"[{'start': 3, 'end': 11, 'username': 'markofu'...","[{'start': 38, 'end': 44, 'probability': 0.866...",NaN,NaN,NaN,NaN,NaN,NaN
21707,8,['2707957786'],2707957786,56187579,2009-07-18T16:19:15.000Z,RT @sans_isc:[Diary]Vulnerability in FireFox 3...,en,2707957786,NaN,0,...,0,0,"[{'start': 3, 'end': 12, 'username': 'sans_isc...","[{'start': 37, 'end': 43, 'probability': 0.837...",NaN,NaN,NaN,NaN,NaN,NaN
21708,9,['2696247609'],2696247609,56187579,2009-07-17T22:02:52.000Z,RT @siliconrepublic Eircom reveals 'cache pois...,en,2696247609,NaN,0,...,0,0,"[{'start': 3, 'end': 19, 'username': 'siliconr...","[{'start': 20, 'end': 25, 'probability': 0.596...",NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
tweet_data_df['id']

0        1005188544216944643
1        1004835128428908546
2        1004470327420825601
3        1004468583794184194
4        1003742842756444160
                ...         
21705             2775715724
21706             2775652740
21707             2707957786
21708             2696247609
21709             2687314573
Name: id, Length: 21710, dtype: int64

we'll check if the 1st 27k coincides with this ones 21k

In [22]:
tweet_data_df_1 = pd.read_excel('tweet_data_df.xlsx')
tweet_data_df_1['id'] = tweet_data_df_1['id'].astype(str)

tweet_data_df_1['conversation_id'] = tweet_data_df_1['conversation_id'].astype(str)

tweet_data_df_1['author_id'] = tweet_data_df_1['author_id'].astype(str)


In [23]:
tweet_data_df['id']

0        1005188544216944643
1        1004835128428908546
2        1004470327420825601
3        1004468583794184194
4        1003742842756444160
                ...         
21705             2775715724
21706             2775652740
21707             2707957786
21708             2696247609
21709             2687314573
Name: id, Length: 21710, dtype: int64

In [25]:
tweet_data_df_1['id'] = tweet_data_df_1['id'].astype('int64')

In [26]:
merged = pd.merge(tweet_data_df_1, tweet_data_df, on = 'id')
merged


,Unnamed: 0_x,id,lang_x,edit_history_tweet_ids_x,text,created_at_x,author_id_x,conversation_id_x,entities.mentions_x,entities.urls_x,...,public_metrics.quote_count_y,public_metrics.impression_count_y,entities.mentions_y,entities.annotations_y,entities.hashtags_y,referenced_tweets_y,geo.place_id_y,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags


In [27]:
tweet_data_df_1.columns

Index(['Unnamed: 0', 'id', 'lang', 'edit_history_tweet_ids', 'text',
       'created_at', 'author_id', 'conversation_id', 'entities.mentions',
       'entities.urls', 'entities.hashtags', 'entities.annotations',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.impression_count', 'referenced_tweets', 'geo.place_id'],
      dtype='object')

In [28]:
tweet_data_df.columns

Index(['Unnamed: 0', 'edit_history_tweet_ids', 'conversation_id', 'author_id',
       'created_at', 'full_text', 'lang', 'id', 'entities.urls',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.impression_count', 'entities.mentions',
       'entities.annotations', 'entities.hashtags', 'referenced_tweets',
       'geo.place_id', 'geo.coordinates.type', 'geo.coordinates.coordinates',
       'entities.cashtags'],
      dtype='object')

In [29]:
tweet_data_df_1.rename(columns={'text': 'full_text'}, inplace=True)


In [30]:
all_data_updated_conv_id = pd.concat([tweet_data_df, tweet_data_df])

all_data_updated_conv_id['id'] = all_data_updated_conv_id['id'].astype(str)
all_data_updated_conv_id['conversation_id'] = all_data_updated_conv_id['conversation_id'].astype(str)
all_data_updated_conv_id['author_id'] = all_data_updated_conv_id['author_id'].astype(str)
all_data_updated_conv_id

,Unnamed: 0,edit_history_tweet_ids,conversation_id,author_id,created_at,full_text,lang,id,entities.urls,public_metrics.retweet_count,...,public_metrics.quote_count,public_metrics.impression_count,entities.mentions,entities.annotations,entities.hashtags,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0,['1005188544216944643'],1005188544216944643,925834897838804992,2018-06-08T20:43:19.000Z,Boletín CSIRT 2018-06-08 https://t.co/30QRd415sD,und,1005188544216944643,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",3,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,['1004835128428908546'],1004835128428908546,925834897838804992,2018-06-07T21:18:58.000Z,Boletín CSIRT 2018-06-07 https://t.co/lBBw4Mtt2d,und,1004835128428908546,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",3,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,['1004470327420825601'],1004470327420825601,925834897838804992,2018-06-06T21:09:22.000Z,Boletín CSIRT 2018-06-06 https://t.co/jYz8n1Gciz,und,1004470327420825601,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",3,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,['1004468583794184194'],1004468583794184194,925834897838804992,2018-06-06T21:02:27.000Z,Update Google Chrome Immediately to Patch a Hi...,en,1004468583794184194,"[{'start': 74, 'end': 97, 'url': 'https://t.co...",1,...,0,0,"[{'start': 102, 'end': 117, 'username': 'TheHa...","[{'start': 7, 'end': 19, 'probability': 0.9137...",NaN,NaN,NaN,NaN,NaN,NaN
4,4,['1003742842756444160'],1003742842756444160,925834897838804992,2018-06-04T20:58:37.000Z,Boletín CSIRT 2018-06-04 https://t.co/kU7OU2LRMA,und,1003742842756444160,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",2,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21705,6,['2775715724'],2775715724,56187579,2009-07-22T10:20:47.000Z,Irish websites compromised with malware code. ...,en,2775715724,NaN,0,...,0,0,NaN,"[{'start': 82, 'end': 84, 'probability': 0.704...",NaN,NaN,NaN,NaN,NaN,NaN
21706,7,['2775652740'],2775652740,56187579,2009-07-22T10:13:14.000Z,RT @markofu: RT @securityninja: OWASP Ireland ...,en,2775652740,NaN,0,...,0,0,"[{'start': 3, 'end': 11, 'username': 'markofu'...","[{'start': 38, 'end': 44, 'probability': 0.866...",NaN,NaN,NaN,NaN,NaN,NaN
21707,8,['2707957786'],2707957786,56187579,2009-07-18T16:19:15.000Z,RT @sans_isc:[Diary]Vulnerability in FireFox 3...,en,2707957786,NaN,0,...,0,0,"[{'start': 3, 'end': 12, 'username': 'sans_isc...","[{'start': 37, 'end': 43, 'probability': 0.837...",NaN,NaN,NaN,NaN,NaN,NaN
21708,9,['2696247609'],2696247609,56187579,2009-07-17T22:02:52.000Z,RT @siliconrepublic Eircom reveals 'cache pois...,en,2696247609,NaN,0,...,0,0,"[{'start': 3, 'end': 19, 'username': 'siliconr...","[{'start': 20, 'end': 25, 'probability': 0.596...",NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
data_labeled = pd.read_csv('data_labeled.csv')
data_labeled

,Unnamed: 0,id,created_at,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,lang,...,hashtags,user_mentions,urls,user_id,screen_name,description,user_location,full_text1,predictions,label
0,0,5.100000e+17,Thu Sep 11 10:19:25 +0000 2014,Collaborazione per potenziare la sicurezza in ...,False,False,False,0,0,it,...,[],[],['http://www.melani.admin.ch/dienstleistungen/...,2.187916e+09,GovCERT_CH,"National Cyber Security Centre (NCSC), Compute...",Bern,collaborazione potenziare sicurezza rete:,0.999537,0.0
1,1,1.310000e+18,Wed Sep 23 16:08:14 +0000 2020,Rilevate nuove versioni dell’email utilizzata ...,False,False,False,1,1,it,...,"['malspam', 'Emotet']",[],['https://csirt.gov.it/contenuti/emotet-indivi...,1.192510e+18,csirt_it,Account ufficiale del Computer Security Incide...,Italia,rilevate nuove versioni dell’email utilizzata ...,0.999387,1.0
2,2,1.330000e+18,Wed Nov 18 16:25:02 +0000 2020,Individuate due vulnerabilità #0day nel softwa...,False,False,False,9,8,it,...,"['0day', 'Cisco', 'RCE']",[],['https://csirt.gov.it/contenuti/vulnerabilita...,1.192510e+18,csirt_it,Account ufficiale del Computer Security Incide...,Italia,individuate vulnerabilità #0day software #cisc...,0.999267,1.0
3,3,1.370000e+18,Wed Mar 03 09:49:20 +0000 2021,Sfruttate 4 vulnerabilità #0day per violare se...,False,False,False,5,10,it,...,"['0day', 'Exchange', 'Microsoft']",[],['https://csirt.gov.it/contenuti/sfruttate-vul...,1.192510e+18,csirt_it,Account ufficiale del Computer Security Incide...,Italia,sfruttate vulnerabilità #0day violare server #...,0.999266,1.0
4,4,1.360000e+18,Mon Feb 08 12:07:57 +0000 2021,Malware #sLoad sfrutta #PEC con allegato malev...,False,False,False,11,18,it,...,"['sLoad', 'PEC']",[],['https://cert-agid.gov.it/news/malware-sload-...,1.273940e+18,AgidCert,Profilo ufficiale del CERT dell'Agenzia per l'...,Roma - Via Liszt 21,malware #sload sfrutta #pec allegato malevolo ...,0.998893,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3279,3279,1.070000e+18,Thu Nov 22 09:37:09 +0000 2018,delighted martijn,NaN,NaN,NaN,1,3,en,...,NaN,NaN,NaN,5.618758e+07,irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,delighted martijn,0.003452,0.0
3280,3280,6.390000e+17,Wed Sep 02 14:45:48 +0000 2015,trend micro продает бизнес китае местной asiainfo,NaN,NaN,NaN,0,0,und,...,NaN,NaN,NaN,1.377952e+08,IlyaShabanov,The founder and CEO of AM Media LLC,Moscow,trend micro продает бизнес китае местной asiainfo,0.002467,0.0
3281,3281,1.460000e+18,Tue Nov 16 21:58:56 +0000 2021,michael,NaN,NaN,NaN,0,2,en,...,NaN,NaN,NaN,5.618758e+07,irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,michael,0.002257,0.0
3282,3282,1.830000e+17,Thu Mar 22 12:47:33 +0000 2012,челябинская модница :)),NaN,NaN,NaN,0,0,ru,...,NaN,NaN,NaN,1.377952e+08,IlyaShabanov,The founder and CEO of AM Media LLC,Moscow,челябинская модница :)),0.002190,0.0


In [64]:
all_data_updated_conv_id.to_excel('all_data_updated_conv_id.xlsx', encoding='utf-f8')

C:\Users\Nastea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [32]:
merged = pd.merge(data_labeled, tweet_data_df_1, on = 'full_text')
merged


,Unnamed: 0_x,id_x,created_at_x,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,lang_x,...,entities.urls,entities.hashtags,entities.annotations,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,referenced_tweets,geo.place_id
0,0,5.100000e+17,Thu Sep 11 10:19:25 +0000 2014,Collaborazione per potenziare la sicurezza in ...,False,False,False,0,0,it,...,"[{'start': 52, 'end': 74, 'url': 'http://t.co/...",NaN,NaN,0,0,0,0,0,NaN,NaN
1,1,1.310000e+18,Wed Sep 23 16:08:14 +0000 2020,Rilevate nuove versioni dell’email utilizzata ...,False,False,False,1,1,it,...,"[{'start': 109, 'end': 132, 'url': 'https://t....","[{'start': 61, 'end': 69, 'tag': 'malspam'}, {...",NaN,1,0,1,0,0,NaN,NaN
2,2,1.330000e+18,Wed Nov 18 16:25:02 +0000 2020,Individuate due vulnerabilità #0day nel softwa...,False,False,False,9,8,it,...,"[{'start': 85, 'end': 108, 'url': 'https://t.c...","[{'start': 30, 'end': 35, 'tag': '0day'}, {'st...",NaN,9,1,8,0,0,NaN,NaN
3,3,1.370000e+18,Wed Mar 03 09:49:20 +0000 2021,Sfruttate 4 vulnerabilità #0day per violare se...,False,False,False,5,10,it,...,"[{'start': 131, 'end': 154, 'url': 'https://t....","[{'start': 26, 'end': 31, 'tag': '0day'}, {'st...",NaN,5,0,10,1,0,NaN,NaN
4,4,1.360000e+18,Mon Feb 08 12:07:57 +0000 2021,Malware #sLoad sfrutta #PEC con allegato malev...,False,False,False,11,18,it,...,"[{'start': 74, 'end': 97, 'url': 'https://t.co...","[{'start': 8, 'end': 14, 'tag': 'sLoad'}, {'st...",NaN,11,0,18,4,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,2658,1.795900e+17,Tue Mar 13 15:28:26 +0000 2012,CERT-GIB at the I-COMference http://t.co/Iw7Fy26z,False,False,False,0,0,en,...,"[{'start': 29, 'end': 49, 'url': 'http://t.co/...",NaN,"[{'start': 17, 'end': 27, 'probability': 0.529...",0,0,0,0,0,NaN,NaN
1135,2664,7.249400e+17,Tue Apr 26 12:35:46 +0000 2016,Never say never again die Zweite https://t.co/...,False,False,True,2,1,en,...,"[{'start': 33, 'end': 56, 'url': 'https://t.co...",NaN,NaN,2,0,1,0,0,"[{'type': 'quoted', 'id': '724918933845872640'}]",NaN
1136,2668,1.460970e+18,Wed Nov 17 14:02:02 +0000 2021,https://t.co/BmwtbRvUWR\n \n𝐂𝐓𝐅 𝐈𝐧𝐭𝐞𝐫𝐧𝐚𝐭𝐢𝐨𝐧𝐚𝐥 ...,False,False,False,1,0,en,...,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN,1,0,0,0,0,NaN,NaN
1137,2670,1.513330e+18,Mon Apr 11 01:30:02 +0000 2022,5 years ago today 🥹 https://t.co/c4uTi8Ubop,False,False,True,4,43,en,...,"[{'start': 20, 'end': 43, 'url': 'https://t.co...",NaN,NaN,4,2,43,0,0,"[{'type': 'quoted', 'id': '851640720293150720'}]",NaN


In [33]:
merged2 = pd.merge(data_labeled, tweet_data_df, on = 'full_text')
merged2

,Unnamed: 0_x,id_x,created_at_x,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,lang_x,...,public_metrics.quote_count,public_metrics.impression_count,entities.mentions,entities.annotations,entities.hashtags,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,148,1.530000e+18,Tue May 17 18:56:09 +0000 2022,Ransomware Conti ataca a Costa Rica https://t....,False,False,False,3,3,it,...,0,0,NaN,NaN,"[{'start': 60, 'end': 74, 'tag': 'Cybersecurit...",NaN,NaN,NaN,NaN,NaN
1,916,3.300000e+17,Fri May 03 02:12:13 +0000 2013,CERT Battalion Coordinator command http://t.co...,False,False,False,2,0,it,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,925,7.550000e+17,Mon Jul 18 10:21:01 +0000 2016,Maxthon Browser Sends Sensitive Data to China ...,False,False,False,1,0,it,...,0,0,"[{'start': 100, 'end': 113, 'username': 'Secur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1580,8.380000e+17,Sat Mar 04 09:12:23 +0000 2017,Radare 2 in 0x1E minutes https://t.co/o0OQCNxo99,False,False,False,10,25,it,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1614,4.400000e+17,Sat Mar 01 00:59:43 +0000 2014,LA DOT INFO \nhttp://t.co/nuQtiie1sP,False,False,False,0,0,it,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2672,9.236770e+17,Thu Oct 26 22:27:23 +0000 2017,Looking forward to seeing you there https://t....,False,False,True,0,2,en,...,0,0,NaN,NaN,NaN,"[{'type': 'quoted', 'id': '923516295068504065'}]",NaN,NaN,NaN,NaN
499,2673,8.241590e+17,Wed Jan 25 07:36:19 +0000 2017,Third round ... 💪 https://t.co/ObpaSUcJNV,False,False,False,2,3,en,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,2674,3.384850e+17,Sun May 26 02:42:16 +0000 2013,test,False,False,False,0,1,en,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,2675,3.872850e+17,Mon Oct 07 18:36:07 +0000 2013,WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,False,False,False,0,0,en,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
d1 = pd.concat([merged,merged2])
d1

,Unnamed: 0_x,id_x,created_at_x,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,lang_x,...,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0,5.100000e+17,Thu Sep 11 10:19:25 +0000 2014,Collaborazione per potenziare la sicurezza in ...,False,False,False,0,0,it,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,1.310000e+18,Wed Sep 23 16:08:14 +0000 2020,Rilevate nuove versioni dell’email utilizzata ...,False,False,False,1,1,it,...,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN
2,2,1.330000e+18,Wed Nov 18 16:25:02 +0000 2020,Individuate due vulnerabilità #0day nel softwa...,False,False,False,9,8,it,...,9,1,8,0,0,NaN,NaN,NaN,NaN,NaN
3,3,1.370000e+18,Wed Mar 03 09:49:20 +0000 2021,Sfruttate 4 vulnerabilità #0day per violare se...,False,False,False,5,10,it,...,5,0,10,1,0,NaN,NaN,NaN,NaN,NaN
4,4,1.360000e+18,Mon Feb 08 12:07:57 +0000 2021,Malware #sLoad sfrutta #PEC con allegato malev...,False,False,False,11,18,it,...,11,0,18,4,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2672,9.236770e+17,Thu Oct 26 22:27:23 +0000 2017,Looking forward to seeing you there https://t....,False,False,True,0,2,en,...,0,0,2,0,0,"[{'type': 'quoted', 'id': '923516295068504065'}]",NaN,NaN,NaN,NaN
499,2673,8.241590e+17,Wed Jan 25 07:36:19 +0000 2017,Third round ... 💪 https://t.co/ObpaSUcJNV,False,False,False,2,3,en,...,2,0,3,0,0,NaN,NaN,NaN,NaN,NaN
500,2674,3.384850e+17,Sun May 26 02:42:16 +0000 2013,test,False,False,False,0,1,en,...,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN
501,2675,3.872850e+17,Mon Oct 07 18:36:07 +0000 2013,WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,False,False,False,0,0,en,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [111]:
d1['full_text1']

0              collaborazione potenziare sicurezza rete:
1      rilevate nuove versioni dell’email utilizzata ...
2      individuate vulnerabilità #0day software #cisc...
3      sfruttate vulnerabilità #0day violare server #...
4      malware #sload sfrutta #pec allegato malevolo ...
                             ...                        
498                                                  NaN
499                                                  NaN
500                                                  NaN
501                                                  NaN
502                                                  NaN
Name: full_text1, Length: 1642, dtype: object

In [69]:
d1.to_excel('d1.xlsx')

In [23]:
d1 = pd.read_excel('d1.xlsx')

In [112]:
d1.reset_index(inplace=True)
d1

,index,Unnamed: 0_x,id_x,created_at_x,full_text,retweeted,favorited,is_quote_status,retweet_count,favorite_count,...,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0,0,5.100000e+17,Thu Sep 11 10:19:25 +0000 2014,Collaborazione per potenziare la sicurezza in ...,False,False,False,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,1,1.310000e+18,Wed Sep 23 16:08:14 +0000 2020,Rilevate nuove versioni dell’email utilizzata ...,False,False,False,1,1,...,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN
2,2,2,1.330000e+18,Wed Nov 18 16:25:02 +0000 2020,Individuate due vulnerabilità #0day nel softwa...,False,False,False,9,8,...,9,1,8,0,0,NaN,NaN,NaN,NaN,NaN
3,3,3,1.370000e+18,Wed Mar 03 09:49:20 +0000 2021,Sfruttate 4 vulnerabilità #0day per violare se...,False,False,False,5,10,...,5,0,10,1,0,NaN,NaN,NaN,NaN,NaN
4,4,4,1.360000e+18,Mon Feb 08 12:07:57 +0000 2021,Malware #sLoad sfrutta #PEC con allegato malev...,False,False,False,11,18,...,11,0,18,4,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,498,2672,9.236770e+17,Thu Oct 26 22:27:23 +0000 2017,Looking forward to seeing you there https://t....,False,False,True,0,2,...,0,0,2,0,0,"[{'type': 'quoted', 'id': '923516295068504065'}]",NaN,NaN,NaN,NaN
1638,499,2673,8.241590e+17,Wed Jan 25 07:36:19 +0000 2017,Third round ... 💪 https://t.co/ObpaSUcJNV,False,False,False,2,3,...,2,0,3,0,0,NaN,NaN,NaN,NaN,NaN
1639,500,2674,3.384850e+17,Sun May 26 02:42:16 +0000 2013,test,False,False,False,0,1,...,0,1,1,0,0,NaN,NaN,NaN,NaN,NaN
1640,501,2675,3.872850e+17,Mon Oct 07 18:36:07 +0000 2013,WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,False,False,False,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [113]:
d1['user_id']

0       2.187916e+09
1       1.192510e+18
2       1.192510e+18
3       1.192510e+18
4       1.273940e+18
            ...     
1637    5.618758e+07
1638    1.538261e+08
1639    2.659454e+07
1640    2.659454e+07
1641    2.659454e+07
Name: user_id, Length: 1642, dtype: float64

In [114]:
d1['tweet_id'] = d1['id_y']

In [115]:
d1 = d1.drop([ 'index', 'Unnamed: 0_x', 'id_x', 'Unnamed: 0_y', 'lang_y', 'user_id', 'created_at_y', 'id_y',
       'retweeted', 'favorited', 'is_quote_status', 'retweet_count',
       'favorite_count',  'in_reply_to_status_id',
       'in_reply_to_user_id', 'geo', 'place', 'coordinates',  'user_id'], axis=1)
d1.columns

Index(['created_at_x', 'full_text', 'lang_x', 'possibly_sensitive', 'hashtags',
       'user_mentions', 'urls', 'screen_name', 'description', 'user_location',
       'full_text1', 'predictions', 'label', 'edit_history_tweet_ids',
       'author_id', 'conversation_id', 'entities.mentions', 'entities.urls',
       'entities.hashtags', 'entities.annotations',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.impression_count', 'referenced_tweets', 'geo.place_id',
       'geo.coordinates.type', 'geo.coordinates.coordinates',
       'entities.cashtags', 'tweet_id'],
      dtype='object')

In [116]:
d1['conversation_id']

0        510009710129741824
1       1308800351508869123
2       1329098299962429441
3       1367049444026429440
4       1358749407622160384
               ...         
1637     923677463938523136
1638     824158943924678656
1639     338485206468157441
1640     387285225861967872
1641     402926942065668096
Name: conversation_id, Length: 1642, dtype: object

In [117]:
# count null values in column 'A'
null_count = d1['predictions'].isnull().sum()

print(null_count)

0


In [118]:
d1['tweet_id'] = d1['tweet_id'].astype(str)
d1['conversation_id'] = d1['conversation_id'].astype(str)
d1['author_id'] = d1['author_id'].astype(str)
d1

,created_at_x,full_text,lang_x,possibly_sensitive,hashtags,user_mentions,urls,screen_name,description,user_location,...,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags,tweet_id
0,Thu Sep 11 10:19:25 +0000 2014,Collaborazione per potenziare la sicurezza in ...,it,False,[],[],['http://www.melani.admin.ch/dienstleistungen/...,GovCERT_CH,"National Cyber Security Centre (NCSC), Compute...",Bern,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,510009710129741824
1,Wed Sep 23 16:08:14 +0000 2020,Rilevate nuove versioni dell’email utilizzata ...,it,False,"['malspam', 'Emotet']",[],['https://csirt.gov.it/contenuti/emotet-indivi...,csirt_it,Account ufficiale del Computer Security Incide...,Italia,...,0,1,0,0,NaN,NaN,NaN,NaN,NaN,1308800351508869123
2,Wed Nov 18 16:25:02 +0000 2020,Individuate due vulnerabilità #0day nel softwa...,it,False,"['0day', 'Cisco', 'RCE']",[],['https://csirt.gov.it/contenuti/vulnerabilita...,csirt_it,Account ufficiale del Computer Security Incide...,Italia,...,1,8,0,0,NaN,NaN,NaN,NaN,NaN,1329098299962429441
3,Wed Mar 03 09:49:20 +0000 2021,Sfruttate 4 vulnerabilità #0day per violare se...,it,False,"['0day', 'Exchange', 'Microsoft']",[],['https://csirt.gov.it/contenuti/sfruttate-vul...,csirt_it,Account ufficiale del Computer Security Incide...,Italia,...,0,10,1,0,NaN,NaN,NaN,NaN,NaN,1367049444026429440
4,Mon Feb 08 12:07:57 +0000 2021,Malware #sLoad sfrutta #PEC con allegato malev...,it,False,"['sLoad', 'PEC']",[],['https://cert-agid.gov.it/news/malware-sload-...,AgidCert,Profilo ufficiale del CERT dell'Agenzia per l'...,Roma - Via Liszt 21,...,0,18,4,0,NaN,NaN,NaN,NaN,NaN,1358749407622160384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,Thu Oct 26 22:27:23 +0000 2017,Looking forward to seeing you there https://t....,en,False,[],[],['https://twitter.com/PsychFriend/status/92351...,irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,...,0,2,0,0,"[{'type': 'quoted', 'id': '923516295068504065'}]",NaN,NaN,NaN,NaN,923677463938523136
1638,Wed Jan 25 07:36:19 +0000 2017,Third round ... 💪 https://t.co/ObpaSUcJNV,en,False,[],[],[],mmorenog,Cyber Threat Intelligence Analyst #Malware #Cy...,yararules.com,...,0,3,0,0,NaN,NaN,NaN,NaN,NaN,824158943924678656
1639,Sun May 26 02:42:16 +0000 2013,test,en,NaN,[],[],[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",...,1,1,0,0,NaN,NaN,NaN,NaN,NaN,338485206468157441
1640,Mon Oct 07 18:36:07 +0000 2013,WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,en,False,[],[],['http://www.youtube.com/watch?v=FbLUsHpJos4'],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,387285225861967872


In [87]:
# d1.to_excel('d1.xlsx', encoding='utf-f8')

C:\Users\Nastea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [119]:
# use groupby and filter to select authors with less than 5 tweets
d1_filtered = d1.groupby('author_id').filter(lambda x: len(x) < 10)

print(d1_filtered)


                        created_at_x  \
261   Thu Oct 30 04:35:45 +0000 2014   
488   Sun Apr 07 15:46:04 +0000 2013   
579   Thu Oct 30 21:52:54 +0000 2014   
620   Wed Sep 23 07:43:49 +0000 2020   
621   Fri May 13 19:53:28 +0000 2022   
...                              ...   
1593  Mon Dec 02 23:57:36 +0000 2013   
1594  Tue Feb 06 15:20:57 +0000 2018   
1608  Thu Jan 05 16:31:40 +0000 2023   
1615  Wed Dec 07 13:09:22 +0000 2022   
1618  Mon Jan 30 17:13:15 +0000 2023   

                                              full_text lang_x  \
261   Rooms, una app alternativa de Facebook a Reddi...     it   
488        An avalanche in Skype http://t.co/ALCXFX9Eun     it   
579   Con Rooms, Facebook le abre la puerta al anoni...     it   
620   If you receive an email from unknown address w...     en   
621   Joint Cyber Security Advisory on Ransomware: h...     en   
...                                                 ...    ...   
1593           What is a botnet? http://t.co/wBJNETLOBO  

In [120]:
num_authors = d1_filtered['author_id'].nunique()
num_authors

16

only 69 tweets goes below criteria less than 10 tw per user

In [121]:
d1_filtered['screen_name']

261       csirtutpl
488         CERTGIB
579       csirtutpl
620         CERT_EE
621         ttcsirt
           ...     
1593    CSIRTPanama
1594        tfcsirt
1608        uuallan
1615        uuallan
1618        uuallan
Name: screen_name, Length: 69, dtype: object

In [122]:
d1_filtered['full_text']

261     Rooms, una app alternativa de Facebook a Reddi...
488          An avalanche in Skype http://t.co/ALCXFX9Eun
579     Con Rooms, Facebook le abre la puerta al anoni...
620     If you receive an email from unknown address w...
621     Joint Cyber Security Advisory on Ransomware: h...
                              ...                        
1593             What is a botnet? http://t.co/wBJNETLOBO
1594    Slowly moving through the lightning talks! No ...
1608    This is a thoughtful piece from @elisabethbraw...
1615                   Well done. https://t.co/NIRmDLICgr
1618                     Thread 👇 https://t.co/xkjrZ95IA3
Name: full_text, Length: 69, dtype: object

In [2]:
d1 = pd.read_excel('d1.xlsx')

Here, df['column_name'].isin(['specific_value_1', 'specific_value_2', 'specific_value_3']) returns a boolean mask that selects all the rows where the 'column_name' column has one of the specified values. The tilde (~) before the condition inverts the boolean mask, selecting all the rows where the 'column_name' does not have any of the specified values.

In [123]:
d1 = d1[~d1['screen_name'].isin(d1_filtered['screen_name'])]
d1

,created_at_x,full_text,lang_x,possibly_sensitive,hashtags,user_mentions,urls,screen_name,description,user_location,...,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags,tweet_id
0,Thu Sep 11 10:19:25 +0000 2014,Collaborazione per potenziare la sicurezza in ...,it,False,[],[],['http://www.melani.admin.ch/dienstleistungen/...,GovCERT_CH,"National Cyber Security Centre (NCSC), Compute...",Bern,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,510009710129741824
1,Wed Sep 23 16:08:14 +0000 2020,Rilevate nuove versioni dell’email utilizzata ...,it,False,"['malspam', 'Emotet']",[],['https://csirt.gov.it/contenuti/emotet-indivi...,csirt_it,Account ufficiale del Computer Security Incide...,Italia,...,0,1,0,0,NaN,NaN,NaN,NaN,NaN,1308800351508869123
2,Wed Nov 18 16:25:02 +0000 2020,Individuate due vulnerabilità #0day nel softwa...,it,False,"['0day', 'Cisco', 'RCE']",[],['https://csirt.gov.it/contenuti/vulnerabilita...,csirt_it,Account ufficiale del Computer Security Incide...,Italia,...,1,8,0,0,NaN,NaN,NaN,NaN,NaN,1329098299962429441
3,Wed Mar 03 09:49:20 +0000 2021,Sfruttate 4 vulnerabilità #0day per violare se...,it,False,"['0day', 'Exchange', 'Microsoft']",[],['https://csirt.gov.it/contenuti/sfruttate-vul...,csirt_it,Account ufficiale del Computer Security Incide...,Italia,...,0,10,1,0,NaN,NaN,NaN,NaN,NaN,1367049444026429440
4,Mon Feb 08 12:07:57 +0000 2021,Malware #sLoad sfrutta #PEC con allegato malev...,it,False,"['sLoad', 'PEC']",[],['https://cert-agid.gov.it/news/malware-sload-...,AgidCert,Profilo ufficiale del CERT dell'Agenzia per l'...,Roma - Via Liszt 21,...,0,18,4,0,NaN,NaN,NaN,NaN,NaN,1358749407622160384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,Thu Oct 26 22:27:23 +0000 2017,Looking forward to seeing you there https://t....,en,False,[],[],['https://twitter.com/PsychFriend/status/92351...,irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,...,0,2,0,0,"[{'type': 'quoted', 'id': '923516295068504065'}]",NaN,NaN,NaN,NaN,923677463938523136
1638,Wed Jan 25 07:36:19 +0000 2017,Third round ... 💪 https://t.co/ObpaSUcJNV,en,False,[],[],[],mmorenog,Cyber Threat Intelligence Analyst #Malware #Cy...,yararules.com,...,0,3,0,0,NaN,NaN,NaN,NaN,NaN,824158943924678656
1639,Sun May 26 02:42:16 +0000 2013,test,en,NaN,[],[],[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",...,1,1,0,0,NaN,NaN,NaN,NaN,NaN,338485206468157441
1640,Mon Oct 07 18:36:07 +0000 2013,WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,en,False,[],[],['http://www.youtube.com/watch?v=FbLUsHpJos4'],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,387285225861967872


check if you filtered out those users who have less than 5 tweets

In [124]:
# use groupby and filter to select authors with less than 5 tweets
d1_filtered = d1.groupby('author_id').filter(lambda x: len(x) < 10)

print(d1_filtered)


Empty DataFrame
Columns: [created_at_x, full_text, lang_x, possibly_sensitive, hashtags, user_mentions, urls, screen_name, description, user_location, full_text1, predictions, label, edit_history_tweet_ids, author_id, conversation_id, entities.mentions, entities.urls, entities.hashtags, entities.annotations, public_metrics.retweet_count, public_metrics.reply_count, public_metrics.like_count, public_metrics.quote_count, public_metrics.impression_count, referenced_tweets, geo.place_id, geo.coordinates.type, geo.coordinates.coordinates, entities.cashtags, tweet_id]
Index: []

[0 rows x 31 columns]


now i would like to create a dataset of 5 tweets per user from the all tweets i have, and try to label. then concatenate them together

In [72]:
# Keep only the top 5 tweets per author_id based on likes_count
tweets_5_per_author = d1.groupby('author_id').apply(lambda x: x.nlargest(5, 'public_metrics.like_count')).reset_index(drop=True)
tweets_5_per_author.shape

(110, 28)

In [73]:
d1['author_id']

0                2187915589
1       1192514152016830464
2       1192514152016830464
3       1192514152016830464
4       1273938586623188992
               ...         
1637               56187579
1638              153826056
1639               26594542
1640               26594542
1641               26594542
Name: author_id, Length: 1573, dtype: object

In [74]:
tweets_5_per_author

,created_at_x,full_text,hashtags,urls,screen_name,description,user_location,full_text1,predictions,label,...,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags,tweet_id
0,Mon Nov 15 21:40:16 +0000 2021,Now things really have gone too far... https:/...,[],[],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,far...,0.002930,0.0,...,66,2216,133,0,NaN,NaN,NaN,NaN,NaN,1460362036924321792
1,Fri Dec 16 15:40:34 +0000 2022,“Account update failed. Description is conside...,[],[],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,“account update failed. description considered...,0.998510,0.0,...,74,1422,64,3334148,NaN,NaN,NaN,NaN,NaN,1603777110937636868
2,Tue Jun 01 09:50:01 +0000 2021,"World’s biggest meat supplier, JBS, suffers cy...",[],['https://grahamcluley.com/worlds-biggest-meat...,gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,"world’s biggest meat supplier, jbs, suffers cy...",0.997222,0.0,...,76,893,49,0,NaN,NaN,NaN,NaN,NaN,1399664524437987328
3,Mon May 17 10:26:29 +0000 2021,Cyberinsurance giant AXA hit by ransomware att...,[],['https://grahamcluley.com/cyberinsurance-gian...,gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,cyberinsurance giant axa ransomware attack cov...,0.998868,0.0,...,11,339,41,0,NaN,NaN,NaN,NaN,NaN,1394237883394019330
4,Mon May 17 21:31:01 +0000 2021,r/t Cyberinsurance giant AXA hit by ransomware...,[],['https://grahamcluley.com/cyberinsurance-gian...,gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,r/t cyberinsurance giant axa ransomware attack...,0.997476,1.0,...,3,65,8,0,NaN,NaN,NaN,NaN,NaN,1394405119014359041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Thu Sep 30 22:18:39 +0000 2021,UPDATE: The installation page for #Flubot has ...,['Flubot'],[],CERTNZ,We're NZ's government authority on #cybersecur...,New Zealand,update: installation #flubot changed warning p...,0.998376,1.0,...,3,246,32,0,NaN,NaN,NaN,NaN,NaN,1443701853665980440
106,Mon Apr 11 01:30:02 +0000 2022,5 years ago today 🥹 https://t.co/c4uTi8Ubop,[],['https://twitter.com/CERTNZ/status/8516407202...,CERTNZ,We're NZ's government authority on #cybersecur...,New Zealand,NaN,0.001145,0.0,...,2,43,0,0,"[{'type': 'quoted', 'id': '851640720293150720'}]",NaN,NaN,NaN,NaN,1513328489466052608
107,Sun Oct 03 23:03:32 +0000 2021,UPDATE: The messaging for the #FluBot malware ...,['FluBot'],[],CERTNZ,We're NZ's government authority on #cybersecur...,New Zealand,update: messaging #flubot malware scam changed...,0.997340,1.0,...,1,38,2,0,NaN,NaN,NaN,NaN,NaN,1444800313118969858
108,Sun Oct 13 19:35:59 +0000 2019,"We've all said it in some shape or form, ""I do...",['getcybersmart'],['http://www.cert.govt.nz/cybersmart'],CERTNZ,We're NZ's government authority on #cybersecur...,New Zealand,"shape form, ""i reckon risk cyber security issu...",0.997020,0.0,...,0,32,3,0,NaN,NaN,NaN,NaN,NaN,1183466430290771968


In [75]:
num_authors = d1['author_id'].nunique()
num_authors

22

In [76]:
num_authors = d1['full_text'].nunique()
num_authors

1573

now concat d1 with other tweets, you'll have to label them

In [125]:
tweets_150_per_author = pd.read_excel('tweets_150_per_author_with_conv_id.xlsx')
tweets_150_per_author

,Unnamed: 0,index,author_id,lang,conversation_id,created_at,edit_history_tweet_ids,id,full_text,entities.urls,...,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.mentions,entities.hashtags,referenced_tweets,geo.place_id,tweet_id
0,0,0,37888607,en,1619227881024131072,2023-01-28T06:56:25.000Z,['1619227881024131072'],1619227881024131072,Verizon admitted to me one of their marketing ...,"[{'start': 257, 'end': 280, 'url': 'https://t....",...,12,2,19,0,4028,NaN,NaN,NaN,NaN,1619227881024131072
1,1,1,137795216,ru,1176602940615909376,2019-09-24T21:02:55.000Z,['1176602940615909376'],1176602940615909376,Опубликовали первое в России масштабное сравне...,"[{'start': 112, 'end': 135, 'url': 'https://t....",...,7,0,19,0,0,"[{'start': 146, 'end': 160, 'username': 'Cisco...",NaN,NaN,NaN,1176602940615909376
2,2,2,137795216,ru,1113834605289852929,2019-04-04T16:03:58.000Z,['1113834605289852929'],1113834605289852929,"ГосСОПКА: что такое, зачем нужна и как устроен...","[{'start': 50, 'end': 73, 'url': 'https://t.co...",...,4,0,9,0,0,"[{'start': 74, 'end': 87, 'username': 'Anti_Ma...",NaN,NaN,NaN,1113834605289852929
3,3,3,137795216,ru,989593024446509056,2018-04-26T19:52:17.000Z,['989593024446509056'],989593024446509056,Круто! Теперь в Gmail на письма можно установи...,"[{'start': 141, 'end': 164, 'url': 'https://t....",...,6,2,9,0,0,NaN,NaN,NaN,NaN,989593024446509056
4,4,4,137795216,ru,1074592431637708800,2018-12-17T09:09:35.000Z,['1074592431637708800'],1074592431637708800,Обзор АПКШ «Континент» 3.9 – многофункциональн...,"[{'start': 94, 'end': 117, 'url': 'https://t.c...",...,2,0,8,0,0,"[{'start': 118, 'end': 131, 'username': 'Anti_...",NaN,NaN,NaN,1074592431637708800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,1565,65,1290574179729891336,pl,1456639341795610624,2021-11-05T15:07:36.000Z,['1456639341795610624'],1456639341795610624,Serwery #GitLab są atakowane #DDoS. Atakujący ...,"[{'start': 179, 'end': 202, 'url': 'https://t....",...,0,0,3,0,0,NaN,"[{'start': 8, 'end': 15, 'tag': 'GitLab'}, {'s...",NaN,NaN,1456639341795610624
1566,1566,66,1290574179729891336,pl,1455842264941568002,2021-11-03T10:20:18.000Z,['1455842264941568002'],1455842264941568002,Firma Google wydała listopadowy biuletyn bezpi...,"[{'start': 257, 'end': 280, 'url': 'https://t....",...,0,0,3,0,0,NaN,"[{'start': 56, 'end': 64, 'tag': 'Android'}]",NaN,NaN,1455842264941568002
1567,1567,67,1290574179729891336,pl,1394182085896777729,2021-05-17T06:44:46.000Z,['1394182085896777729'],1394182085896777729,"Płk Łukasz Jędrzejczak, zastępca dyrektora NCB...","[{'start': 179, 'end': 202, 'url': 'https://t....",...,0,0,2,0,0,NaN,NaN,NaN,NaN,1394182085896777729
1568,1568,68,1290574179729891336,pl,1470695622881812480,2021-12-14T10:02:15.000Z,['1470695622881812480'],1470695622881812480,Najnowsza aktualizacja dla produktów Apple - i...,"[{'start': 78, 'end': 101, 'url': 'https://t.c...",...,0,0,1,0,0,NaN,"[{'start': 230, 'end': 236, 'tag': 'apple'}, {...",NaN,NaN,1470695622881812480


In [126]:
tweets_150_per_author['lang']

0       en
1       ru
2       ru
3       ru
4       ru
        ..
1565    pl
1566    pl
1567    pl
1568    pl
1569    pl
Name: lang, Length: 1570, dtype: object

In [127]:
d1.columns

Index(['created_at_x', 'full_text', 'lang_x', 'possibly_sensitive', 'hashtags',
       'user_mentions', 'urls', 'screen_name', 'description', 'user_location',
       'full_text1', 'predictions', 'label', 'edit_history_tweet_ids',
       'author_id', 'conversation_id', 'entities.mentions', 'entities.urls',
       'entities.hashtags', 'entities.annotations',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.impression_count', 'referenced_tweets', 'geo.place_id',
       'geo.coordinates.type', 'geo.coordinates.coordinates',
       'entities.cashtags', 'tweet_id'],
      dtype='object')

In [128]:
data_to_u2v = pd.concat([tweets_150_per_author, d1])
data_to_u2v

,Unnamed: 0,index,author_id,lang,conversation_id,created_at,edit_history_tweet_ids,id,full_text,entities.urls,...,urls,screen_name,description,user_location,full_text1,predictions,label,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0.0,0.0,37888607,en,1619227881024131072,2023-01-28T06:56:25.000Z,['1619227881024131072'],1.619228e+18,Verizon admitted to me one of their marketing ...,"[{'start': 257, 'end': 280, 'url': 'https://t....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,137795216,ru,1176602940615909376,2019-09-24T21:02:55.000Z,['1176602940615909376'],1.176603e+18,Опубликовали первое в России масштабное сравне...,"[{'start': 112, 'end': 135, 'url': 'https://t....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2.0,137795216,ru,1113834605289852929,2019-04-04T16:03:58.000Z,['1113834605289852929'],1.113835e+18,"ГосСОПКА: что такое, зачем нужна и как устроен...","[{'start': 50, 'end': 73, 'url': 'https://t.co...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,137795216,ru,989593024446509056,2018-04-26T19:52:17.000Z,['989593024446509056'],9.895930e+17,Круто! Теперь в Gmail на письма можно установи...,"[{'start': 141, 'end': 164, 'url': 'https://t....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,137795216,ru,1074592431637708800,2018-12-17T09:09:35.000Z,['1074592431637708800'],1.074592e+18,Обзор АПКШ «Континент» 3.9 – многофункциональн...,"[{'start': 94, 'end': 117, 'url': 'https://t.c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,NaN,NaN,56187579,NaN,923677463938523136,NaN,['923677463938523136'],NaN,Looking forward to seeing you there https://t....,"[{'start': 36, 'end': 59, 'url': 'https://t.co...",...,['https://twitter.com/PsychFriend/status/92351...,irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,NaN,0.001145,0.0,NaN,NaN,NaN
1638,NaN,NaN,153826056,NaN,824158943924678656,NaN,['824158943924678656'],NaN,Third round ... 💪 https://t.co/ObpaSUcJNV,"[{'start': 18, 'end': 41, 'url': 'https://t.co...",...,[],mmorenog,Cyber Threat Intelligence Analyst #Malware #Cy...,yararules.com,NaN,0.001145,0.0,NaN,NaN,NaN
1639,NaN,NaN,26594542,NaN,338485206468157441,NaN,['338485206468157441'],NaN,test,NaN,...,[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN
1640,NaN,NaN,26594542,NaN,387285225861967872,NaN,['387285225861967872'],NaN,WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,"[{'start': 32, 'end': 54, 'url': 'http://t.co/...",...,['http://www.youtube.com/watch?v=FbLUsHpJos4'],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN


In [129]:
data_to_u2v.columns

Index(['Unnamed: 0', 'index', 'author_id', 'lang', 'conversation_id',
       'created_at', 'edit_history_tweet_ids', 'id', 'full_text',
       'entities.urls', 'entities.annotations', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'public_metrics.impression_count',
       'entities.mentions', 'entities.hashtags', 'referenced_tweets',
       'geo.place_id', 'tweet_id', 'created_at_x', 'lang_x',
       'possibly_sensitive', 'hashtags', 'user_mentions', 'urls',
       'screen_name', 'description', 'user_location', 'full_text1',
       'predictions', 'label', 'geo.coordinates.type',
       'geo.coordinates.coordinates', 'entities.cashtags'],
      dtype='object')

In [130]:
data_to_u2v = data_to_u2v.drop([ 'Unnamed: 0', 'index','id'], axis=1)
data_to_u2v.reset_index(inplace=True, drop=True)
data_to_u2v

,author_id,lang,conversation_id,created_at,edit_history_tweet_ids,full_text,entities.urls,entities.annotations,public_metrics.retweet_count,public_metrics.reply_count,...,urls,screen_name,description,user_location,full_text1,predictions,label,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,37888607,en,1619227881024131072,2023-01-28T06:56:25.000Z,['1619227881024131072'],Verizon admitted to me one of their marketing ...,"[{'start': 257, 'end': 280, 'url': 'https://t....","[{'start': 0, 'end': 6, 'probability': 0.6532,...",12,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,137795216,ru,1176602940615909376,2019-09-24T21:02:55.000Z,['1176602940615909376'],Опубликовали первое в России масштабное сравне...,"[{'start': 112, 'end': 135, 'url': 'https://t....",NaN,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,137795216,ru,1113834605289852929,2019-04-04T16:03:58.000Z,['1113834605289852929'],"ГосСОПКА: что такое, зачем нужна и как устроен...","[{'start': 50, 'end': 73, 'url': 'https://t.co...",NaN,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,137795216,ru,989593024446509056,2018-04-26T19:52:17.000Z,['989593024446509056'],Круто! Теперь в Gmail на письма можно установи...,"[{'start': 141, 'end': 164, 'url': 'https://t....",NaN,6,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,137795216,ru,1074592431637708800,2018-12-17T09:09:35.000Z,['1074592431637708800'],Обзор АПКШ «Континент» 3.9 – многофункциональн...,"[{'start': 94, 'end': 117, 'url': 'https://t.c...",NaN,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56187579,NaN,923677463938523136,NaN,['923677463938523136'],Looking forward to seeing you there https://t....,"[{'start': 36, 'end': 59, 'url': 'https://t.co...",NaN,0,0,...,['https://twitter.com/PsychFriend/status/92351...,irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,NaN,0.001145,0.0,NaN,NaN,NaN
3139,153826056,NaN,824158943924678656,NaN,['824158943924678656'],Third round ... 💪 https://t.co/ObpaSUcJNV,"[{'start': 18, 'end': 41, 'url': 'https://t.co...",NaN,2,0,...,[],mmorenog,Cyber Threat Intelligence Analyst #Malware #Cy...,yararules.com,NaN,0.001145,0.0,NaN,NaN,NaN
3140,26594542,NaN,338485206468157441,NaN,['338485206468157441'],test,NaN,NaN,0,1,...,[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN
3141,26594542,NaN,387285225861967872,NaN,['387285225861967872'],WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,"[{'start': 32, 'end': 54, 'url': 'http://t.co/...",NaN,0,0,...,['http://www.youtube.com/watch?v=FbLUsHpJos4'],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN


In [132]:
d1['lang_x']

0       it
1       it
2       it
3       it
4       it
        ..
1637    en
1638    en
1639    en
1640    en
1641    en
Name: lang_x, Length: 1573, dtype: object

In [147]:
data_to_u2v.columns

Index(['author_id', 'lang', 'conversation_id', 'created_at',
       'edit_history_tweet_ids', 'full_text', 'entities.urls',
       'entities.annotations', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'public_metrics.impression_count',
       'entities.mentions', 'entities.hashtags', 'referenced_tweets',
       'geo.place_id', 'tweet_id', 'possibly_sensitive', 'hashtags',
       'user_mentions', 'screen_name', 'description', 'user_location',
       'full_text1', 'predictions', 'label', 'geo.coordinates.type',
       'geo.coordinates.coordinates', 'entities.cashtags'],
      dtype='object')

In [141]:
data_to_u2v['lang'] = data_to_u2v['lang'].combine_first(data_to_u2v['lang_x']).fillna('')
data_to_u2v = data_to_u2v.drop(columns=['lang_x'])
# count null values in column 'A'
null_count = data_to_u2v['lang'].isnull().sum()

print(null_count)

0


In [143]:
data_to_u2v['created_at'] = data_to_u2v['created_at'].combine_first(data_to_u2v['created_at_x']).fillna('')
data_to_u2v = data_to_u2v.drop(columns=['created_at_x'])

# count null values in column 'A'
null_count = data_to_u2v['lang'].isnull().sum()

print(null_count)

0


In [145]:
data_to_u2v['entities.urls'] = data_to_u2v['entities.urls'].combine_first(data_to_u2v['urls']).fillna('')
data_to_u2v = data_to_u2v.drop(columns=['urls'])

# count null values in column 'A'
null_count = data_to_u2v['entities.urls'].isnull().sum()

print(null_count)

0


In [149]:
data_to_u2v['tweet_id'] = data_to_u2v['tweet_id'].astype(str)
data_to_u2v['conversation_id'] = data_to_u2v['conversation_id'].astype(str)
data_to_u2v['author_id'] = data_to_u2v['author_id'].astype(str)


In [151]:
data_to_u2v.to_excel('data_to_u2v_2023_3_24.xlsx', encoding='utf-f8')

C:\Users\Nastea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [3]:
data_to_u2v = pd.read_excel('data_to_u2v_2023_3_24.xlsx')

In [4]:
# count null values in column 'full_text'
null_count = data_to_u2v['full_text'].isnull().sum()
print(null_count)

0


In [2]:
data_to_u2v = pd.read_excel('data_to_u2v_2023_3_24.xlsx')

In [6]:
data_to_u2v

,Unnamed: 0,author_id,lang,conversation_id,created_at,edit_history_tweet_ids,full_text,entities.urls,entities.annotations,public_metrics.retweet_count,...,user_mentions,screen_name,description,user_location,full_text1,predictions,label,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0,37888607,en,1619227881024131072,2023-01-28T06:56:25.000Z,['1619227881024131072'],Verizon admitted to me one of their marketing ...,"[{'start': 257, 'end': 280, 'url': 'https://t....","[{'start': 0, 'end': 6, 'probability': 0.6532,...",12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,137795216,ru,1176602940615909376,2019-09-24T21:02:55.000Z,['1176602940615909376'],Опубликовали первое в России масштабное сравне...,"[{'start': 112, 'end': 135, 'url': 'https://t....",NaN,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,137795216,ru,1113834605289852929,2019-04-04T16:03:58.000Z,['1113834605289852929'],"ГосСОПКА: что такое, зачем нужна и как устроен...","[{'start': 50, 'end': 73, 'url': 'https://t.co...",NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,137795216,ru,989593024446509056,2018-04-26T19:52:17.000Z,['989593024446509056'],Круто! Теперь в Gmail на письма можно установи...,"[{'start': 141, 'end': 164, 'url': 'https://t....",NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,137795216,ru,1074592431637708800,2018-12-17T09:09:35.000Z,['1074592431637708800'],Обзор АПКШ «Континент» 3.9 – многофункциональн...,"[{'start': 94, 'end': 117, 'url': 'https://t.c...",NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,56187579,en,923677463938523136,Thu Oct 26 22:27:23 +0000 2017,['923677463938523136'],Looking forward to seeing you there https://t....,"[{'start': 36, 'end': 59, 'url': 'https://t.co...",NaN,0,...,[],irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,NaN,0.001145,0.0,NaN,NaN,NaN
3139,3139,153826056,en,824158943924678656,Wed Jan 25 07:36:19 +0000 2017,['824158943924678656'],Third round ... 💪 https://t.co/ObpaSUcJNV,"[{'start': 18, 'end': 41, 'url': 'https://t.co...",NaN,2,...,[],mmorenog,Cyber Threat Intelligence Analyst #Malware #Cy...,yararules.com,NaN,0.001145,0.0,NaN,NaN,NaN
3140,3140,26594542,en,338485206468157441,Sun May 26 02:42:16 +0000 2013,['338485206468157441'],test,[],NaN,0,...,[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN
3141,3141,26594542,en,387285225861967872,Mon Oct 07 18:36:07 +0000 2013,['387285225861967872'],WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,"[{'start': 32, 'end': 54, 'url': 'http://t.co/...",NaN,0,...,[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN


In [6]:
# count the number of tweets for each author
counts = data_to_u2v['author_id'].value_counts()

# filter the results to show only authors with less than 20 tweets
less_than_20 = counts[counts < 20]

# print the authors with less than 20 tweets
print(less_than_20)

601041417     18
26594542      17
153826056     15
495814279     14
2364250093    13
126727783     13
2787968336    11
37888607       1
Name: author_id, dtype: int64


To select 20 tweets of each author who has posted at least 25 tweets and save them in a new dataset, you can use Pandas' groupby() function to group the tweets by author, count the number of tweets for each author, filter the authors who have posted at least 25 tweets, and then select 20 tweets for each author using the head() function. Here's an example:

In [7]:
# group the tweets by author and count the number of tweets for each author
counts = data_to_u2v.groupby('author_id').size()
# filter the authors who have posted at least 25 tweets
authors_with_25_tweets = counts[counts >= 25]

In [8]:
# select 20 tweets for each author and save them in a new dataset
new_data = []
for author_id in authors_with_25_tweets.index:
    author_tweets = data_to_u2v[data_to_u2v['author_id'] == author_id].head(20)
    new_data.append(author_tweets)




In [11]:
u2v_test = pd.concat(new_data)
u2v_test

,Unnamed: 0,author_id,lang,conversation_id,created_at,edit_history_tweet_ids,full_text,entities.urls,entities.annotations,public_metrics.retweet_count,...,user_mentions,screen_name,description,user_location,full_text1,predictions,label,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
2668,2668,11791512,ro,1407658084814708736,Wed Jun 23 11:13:35 +0000 2021,['1407658084814708736'],Random recipe password generator. https://t.co...,"[{'start': 34, 'end': 57, 'url': 'https://t.co...",NaN,5,...,[],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,random recipe password generator.,0.997680,1.0,NaN,NaN,NaN
2672,2672,11791512,ro,1470868957066375173,Tue Dec 14 21:31:01 +0000 2021,['1470868957066375173'],Police arrest suspected ransomware affiliate m...,"[{'start': 63, 'end': 86, 'url': 'https://t.co...",NaN,1,...,[],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,police arrest suspected ransomware affiliate r...,0.988512,0.0,NaN,NaN,NaN
2684,2684,11791512,en,1390254036012703747,Thu May 06 10:36:06 +0000 2021,['1390254036012703747'],"When you say “leading figures”, do you mean th...","[{'start': 154, 'end': 177, 'url': 'https://t....",NaN,15,...,[],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,"“leading figures”, power seek weaken secure en...",0.999541,0.0,NaN,NaN,NaN
2693,2693,11791512,en,1561690250078224384,Mon Aug 22 12:22:25 +0000 2022,['1561690250078224384'],"Mac users urged to update Zoom, after security...","[{'start': 101, 'end': 124, 'url': 'https://t....","[{'start': 0, 'end': 2, 'probability': 0.9086,...",13,...,[],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,"user urged update zoom, security patch release...",0.999313,1.0,NaN,NaN,NaN
2704,2704,11791512,en,1617203522977644544,Sun Jan 22 16:52:20 +0000 2023,['1617203522977644544'],Thanks to the @SecurEnvoy team who have been s...,"[{'start': 255, 'end': 278, 'url': 'https://t....","[{'start': 139, 'end': 141, 'probability': 0.5...",0,...,['SecurEnvoy'],gcluley,Award-winning computer security keynote speake...,@gcluley@mastodon.green,"@securenvoy team sponsoring week, supporting i...",0.999148,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,1542,1290574179729891336,pl,1572501963421650944,2022-09-21T08:24:18.000Z,['1572501963421650944'],Zapraszamy do zapoznania się z najnowszą publi...,"[{'start': 133, 'end': 156, 'url': 'https://t....",NaN,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1543,1543,1290574179729891336,pl,1501537347187658755,2022-03-09T12:36:15.000Z,['1501537347187658755'],Informacje dotyczące zagrożenia #HermeticWiper...,"[{'start': 57, 'end': 80, 'url': 'https://t.co...",NaN,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1544,1544,1290574179729891336,pl,1509799545479254024,2022-04-01T07:47:17.000Z,['1509799545479254024'],Dostępna jest nowa wersja #iOS - 15.4.1. Aktua...,NaN,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1545,1545,1290574179729891336,pl,1356574326133297152,2021-02-02T12:04:57.000Z,['1356574326133297152'],Cyber Threat Intelligence. Część druga.\n\nW o...,"[{'start': 243, 'end': 266, 'url': 'https://t....",NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
u2v_test['tweet_id'] = u2v_test['tweet_id'].astype(str)
u2v_test['conversation_id'] = u2v_test['conversation_id'].astype(str)
u2v_test['author_id'] = u2v_test['author_id'].astype(str)

In [14]:
u2v_test.to_excel('u2v_test.xlsx', encoding='utf-f8')

C:\Users\Nastea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
u2v_test

no this is not good, we should first see that users have at least 10 tweets, then to substract them....theoretically nobody should have tweets less than 10, but for those users that have 10 you cannot take all of them

check a method to take from those that have at least 15 tweets


In [16]:
# delete the rows in the original dataframe that match the rows in the new dataframe
mask = data_to_u2v.isin(u2v_test)
data_to_u2v = data_to_u2v[~mask.all(axis=1)]

# print the updated dataframe
data_to_u2v

,Unnamed: 0,author_id,lang,conversation_id,created_at,edit_history_tweet_ids,full_text,entities.urls,entities.annotations,public_metrics.retweet_count,...,user_mentions,screen_name,description,user_location,full_text1,predictions,label,geo.coordinates.type,geo.coordinates.coordinates,entities.cashtags
0,0,37888607,en,1619227881024131072,2023-01-28T06:56:25.000Z,['1619227881024131072'],Verizon admitted to me one of their marketing ...,"[{'start': 257, 'end': 280, 'url': 'https://t....","[{'start': 0, 'end': 6, 'probability': 0.6532,...",12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,137795216,ru,1176602940615909376,2019-09-24T21:02:55.000Z,['1176602940615909376'],Опубликовали первое в России масштабное сравне...,"[{'start': 112, 'end': 135, 'url': 'https://t....",NaN,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,137795216,ru,1113834605289852929,2019-04-04T16:03:58.000Z,['1113834605289852929'],"ГосСОПКА: что такое, зачем нужна и как устроен...","[{'start': 50, 'end': 73, 'url': 'https://t.co...",NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,137795216,ru,989593024446509056,2018-04-26T19:52:17.000Z,['989593024446509056'],Круто! Теперь в Gmail на письма можно установи...,"[{'start': 141, 'end': 164, 'url': 'https://t....",NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,137795216,ru,1074592431637708800,2018-12-17T09:09:35.000Z,['1074592431637708800'],Обзор АПКШ «Континент» 3.9 – многофункциональн...,"[{'start': 94, 'end': 117, 'url': 'https://t.c...",NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,56187579,en,923677463938523136,Thu Oct 26 22:27:23 +0000 2017,['923677463938523136'],Looking forward to seeing you there https://t....,"[{'start': 36, 'end': 59, 'url': 'https://t.co...",NaN,0,...,[],irisscert,IRISS-CERT is Ireland's first CSIRT (Computer ...,Dublin,NaN,0.001145,0.0,NaN,NaN,NaN
3139,3139,153826056,en,824158943924678656,Wed Jan 25 07:36:19 +0000 2017,['824158943924678656'],Third round ... 💪 https://t.co/ObpaSUcJNV,"[{'start': 18, 'end': 41, 'url': 'https://t.co...",NaN,2,...,[],mmorenog,Cyber Threat Intelligence Analyst #Malware #Cy...,yararules.com,NaN,0.001145,0.0,NaN,NaN,NaN
3140,3140,26594542,en,338485206468157441,Sun May 26 02:42:16 +0000 2013,['338485206468157441'],test,[],NaN,0,...,[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN
3141,3141,26594542,en,387285225861967872,Mon Oct 07 18:36:07 +0000 2013,['387285225861967872'],WHAT TO DO IF THERE WAS A FIRE \nhttp://t.co/x...,"[{'start': 32, 'end': 54, 'url': 'http://t.co/...",NaN,0,...,[],CertLAFD,Los Angeles Fire Department CERT Program lafdc...,"Los Angeles, CA",NaN,0.001145,0.0,NaN,NaN,NaN


In [18]:
data_to_u2v['tweet_id'] = data_to_u2v['tweet_id'].astype(str)
data_to_u2v['conversation_id'] = data_to_u2v['conversation_id'].astype(str)
data_to_u2v['author_id'] = data_to_u2v['author_id'].astype(str)

In [19]:
data_to_u2v.to_excel('u2v_train.xlsx', encoding='utf-f8')